In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import warnings
warnings.simplefilter("ignore")

In [2]:
list_num_user = [1000]
list_exp_type = ["fair"]
path = "artificial_insertion"
path_extra = path+"_extra"
exp_type = "fair"
num_user = 1000

plotting_dict = {"x":"position", "y":"score", "hue":"measure", "palette":"colorblind","markersize":7}

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 400


In [3]:
def clean_eff_and_rel(k):
    k = k.str.strip("@10")
    k = k.str.replace("_our","")
    k = k.str.replace("_ori","")
    return k

def modify(lab):
    if lab in ["IBO$_{ori}$", "IBO$_{our}$","Jain", "FSat", "QF", "Ent"]:
        lab = "$\\uparrow$"+lab
    elif lab in ["IWO$_{ori}$", "IWO$_{our}$", "Gini","HD"]:
        if lab == "HD":
            lab += "$_{ori}$"
        lab = "$\\downarrow$"+lab
    return lab

def set_ticks(axes):
    for this_ax in axes.values():
        this_ax.set_xlabel("fraction of artificially inserted items")
        this_ax.set(xticks=np.linspace(0, 1, num=11))
        this_ax.tick_params(axis='x', which='major', labelsize=7.5, rotation=0)

In [ ]:
fig, axes = plt.subplot_mosaic("AD", figsize=(6,3.35),sharey=True)
with open(f"{path}/{exp_type}_user_{str(num_user).zfill(4)}_exact_km.pickle", "rb") as f:
    result = pickle.load(f)

df = pd.DataFrame(result)
df.columns = df.columns.astype(int) / 10
df.index = clean_eff_and_rel(df.index)
df = df.unstack().reset_index()
df.columns = ["position","measure","score"]
df = df.set_index("measure")

rel_measures = ['HR', 'MRR', 'P', 'MAP', 'R', 'NDCG']
fair_measures = ['Jain', 'QF', 'Ent', 'FSat', 'Gini']

thisdata = df

rel_result = thisdata.loc[rel_measures].reset_index()
fair_result = thisdata.loc[fair_measures].reset_index()

sns.lineplot(data=rel_result, ax=axes["A"], style="measure", markers=["","",4,"", 5,""], **plotting_dict)
sns.lineplot(data=fair_result, ax=axes["D"], style="measure",  markers=["",4,5,6,""], **plotting_dict)

handles, ori_labels = axes["D"].get_legend_handles_labels()
new_labels = [modify(lab) for lab in ori_labels]

kw_args = dict(ncols=3,columnspacing=1, fontsize=8, handletextpad=0.2,loc="upper center", bbox_to_anchor=(0.5, 1.37))

axes["A"].legend(title="$\\uparrow$EFF", **kw_args)     
axes["D"].legend(handles, new_labels, title=f"FAIR", **kw_args)     

set_ticks(axes)

fig.tight_layout()    
fig.show()
    
now = datetime.now()
time = str(now.strftime("%Y-%m-%d_%H%M%S"))
fig.savefig(f"{path_extra}/insert_eff_fair_{time}.pdf",bbox_inches="tight")   

In [4]:
def clean_joint(k):
    k = k.str.strip("@10")
    k = k.str.replace("_true","")
    k = k.str.replace("_div","$_{\div}$", regex=False)
    k = k.str.replace("_mul","$_{\\times}$", regex=False)
    k = k.str.replace("_ori","$_{ori}$", regex=False)
    k = k.str.replace("_our","$_{our}$", regex=False)
    k = k.str.replace("-unfixed-att","$_{ori\ e}$", regex=False)
    k = k.str.replace("-cut-off","$_{\ J}$", regex=False)
    return k

def forward(x):
    return x**(1/6)

def inverse(x):
    return x**6 * np.sign(x)

In [ ]:
fig, axes = plt.subplot_mosaic("EFB", figsize=(9.5,3.35))
with open(f"{path}/{exp_type}_user_{str(num_user).zfill(4)}_exact_km.pickle", "rb") as f:
    result = pickle.load(f)
df = pd.DataFrame(result)
df.columns = df.columns.astype(int) / 10
df.index = clean_joint(df.index)
df = df.unstack().reset_index()
df.columns = ["position","measure","score"]
df = df.set_index("measure")

thisdata = df

joint_group3 = ['IAA$_{ori}$', 'MME$_{ori}$','IFD$_{\div}$','II-F$_{ori}$', 'IFD$_{\\times}$', 'AI-F$_{ori}$']
joint_group1 = ["IBO$_{ori}$", "IWO$_{ori}$", "IBO$_{our}$", "IWO$_{our}$","HD"]

joint_group3_result = thisdata.loc[joint_group3].reset_index()
joint_group1_result = thisdata.loc[joint_group1].reset_index()

sns.lineplot(data=joint_group3_result, ax=axes["E"], style="measure", markers=[5,"","", 4, "",6], mew=0.4,**plotting_dict)
handles, ori_labels = axes["E"].get_legend_handles_labels()
new_labels = [modify(lab).replace("$_{ori}$","") for lab in ori_labels]
axes["E"].legend(handles, new_labels, title="$\\downarrow$JOINT (ori)", ncols=3,columnspacing=1, fontsize=8, handletextpad=0.2,loc="upper center", bbox_to_anchor=(0.55, 1.375))     
axes["E"].set_yscale('function', functions=(forward,inverse))
axes["E"].set_yticks([0, 10**-4, 10**-3, 10**-2, 0.1])
axes["E"].ticklabel_format(axis='y', style='sci', scilimits=(0,0), useMathText=True)
axes["E"].yaxis.get_offset_text().set_fontsize(8)

sns.lineplot(data=joint_group1_result, ax=axes["B"], style="measure", markers=[4,6,5,7,""], **plotting_dict)
handles, ori_labels = axes["B"].get_legend_handles_labels()
new_labels = [modify(lab) for lab in ori_labels]
axes["B"].legend(handles, new_labels,title=f"JOINT", ncols=3,columnspacing=1, fontsize=8, handletextpad=0.08,loc="upper center", bbox_to_anchor=(0.5, 1.375))    


with open(f"{path_extra}/{exp_type}_user_{str(num_user).zfill(4)}_exact_km.pickle", "rb") as f:
    result_extra = pickle.load(f)

df = pd.DataFrame(result_extra)
df.columns = df.columns.astype(int) / 10
df.index = clean_joint(df.index)
df = df.unstack().reset_index()
df.columns = ["position","measure","score"]
df = df.sort_values(by=["position","measure"])
df = df.set_index(["measure", "position"])


colors = sns.color_palette("colorblind")[:6]

new_plot_dict = {"x":"position", "y":"score", "hue":"measure", "markersize":7}

selected_index = ["IAA$_{our}$", 'IAA$_{our}$$_{ori\ e}$',   'IFD$_{\div}$$_{our}$', 'II-F$_{our}$', 
                    'IFD$_{\\times}$$_{our}$',]
df = df.reindex(selected_index, level=0).reset_index(level=1)

sns.lineplot(data=df, ax=axes["F"], style="measure", markers = [5,7,"",4,""], palette=colors, **new_plot_dict)
handles, ori_labels = axes["F"].get_legend_handles_labels()
new_labels = [lab.replace("$_{our}$","") for lab in ori_labels]
axes["F"].legend(handles, new_labels,title=f"$\\downarrow$JOINT (our)", ncols=3,columnspacing=1, fontsize=8, handletextpad=0.2,loc="upper center", bbox_to_anchor=(0.5, 1.375))    


set_ticks(axes)

fig.tight_layout()    
fig.show()
    
now = datetime.now()
time = str(now.strftime("%Y-%m-%d_%H%M%S"))
fig.savefig(f"{path_extra}/insert_joint_{time}.pdf",bbox_inches="tight")   